# ОБРАБОТКА ДАННЫХ:
**Описание:** Были скачаны файлы со статистикой по различным тематикам сообществ и был реализован алгоритм в Python, в котором считываются файлы из директории и обрабатываются с целью формирования матрицы объекты-признаки.
## 1. Инициализация первичных данных из файлов со статистикой

In [14]:
import pandas as pd
import glob
import numpy as np
import os.path
import random
from collections import defaultdict
from datetime import datetime
import time

tegs = ['Дни недели','Время суток', 'Лайки','Репосты','Комментарии','Просмотры','ER', 'Подписчики',
'Дни недели/объем текста', 'Время суток/объем текста','Количество по длине текста',
'ER по длине текста','Количество по типу контента','ER по типу контента', 'Количество публикаций',
'Хештеги','Хештеги/ER', 'Относительная активность дни недели/объем текста',
'Относительная активность время суток/объем текста', 'Относительная активность по типу контента',
'Относительная активность по длине текста', 'Относительная активность по хештегам']
tegs_pd = pd.DataFrame(tegs)

tegs_extra = ['Относительная активность дни недели/объем текста',
'Относительная активность время суток/объем текста',]
tegs_extra_pd = pd.DataFrame(tegs_extra)

tegs_full = ['Сравнительная таблица','Дни недели','Время суток', 'Лайки','Репосты','Комментарии',
'Просмотры','ER', 'Подписчики','Дни недели/объем текста', 'Время суток/объем текста',
'Количество по длине текста', 'ER по длине текста','Количество по типу контента','ER по типу контента',
'Количество публикаций','Хештеги','Хештеги/ER', 'Относительная активность дни недели/объем текста',
'Относительная активность время суток/объем текста', 'Относительная активность по типу контента',
'Относительная активность по длине текста', 'Относительная активность по хештегам']
tegs_full_pd = pd.DataFrame(tegs_full)
m = 0;n = 0
#names = ['df_1','df_2','df_3', 'df_4', 'df_5', 'df_6', 'df_7', 'df_8', 'df_9', 'df_10', 'df_11', 'df_12', 'df_13',
         #'df_14', 'df_15', 'df_16', 'df_17', 'df_18', 'df_19', 'df_20', 'df_21', 'df_22', 'df_23']
#d = dict.fromkeys(names, pd.DataFrame())
df_1=pd.DataFrame();df_2=pd.DataFrame();df_3=pd.DataFrame();df_4=pd.DataFrame();df_5=pd.DataFrame();df_6=pd.DataFrame()
df_7=pd.DataFrame();df_8=pd.DataFrame();df_9=pd.DataFrame();df_10=pd.DataFrame();df_11=pd.DataFrame();df_12=pd.DataFrame()
df_13=pd.DataFrame();df_14=pd.DataFrame();df_15=pd.DataFrame();df_16=pd.DataFrame();df_17=pd.DataFrame();df_18=pd.DataFrame()
df_19=pd.DataFrame();df_20=pd.DataFrame();df_21=pd.DataFrame();df_22=pd.DataFrame()#df_23=pd.DataFrame()

day_week = ['Пн','Вт','Ср','Чт','Пт','Сб','Вс']
day_week_pd = pd.DataFrame(day_week).T

def record_excel(df,str):
    writer = pd.ExcelWriter(str + '.xlsx', engine='xlsxwriter')
    df.to_excel(writer, sheet_name='Sheet2')
    writer.save()

## 2. Формирование фреймов данных по характеристикам (признакам)

In [15]:
start_time1 = datetime.now()
path = 'D:\Projects_for_my_science_work\Article_1_Clustering\data_of_community\data_vk_2021_2022\data_4000\every_data_4000\every_data_1000_750_1000'
num_files = len([f for f in os.listdir(path)
                if os.path.isfile(os.path.join(path, f))])
print('Files of directory:',num_files)
print('\n\n')


count_file = 0
for f in glob.glob('D:\Projects_for_my_science_work\Article_1_Clustering\data_of_community\data_vk_2021_2022\data_4000\every_data_4000\every_data_1000_750_1000\*.xlsx'):
    #print(f)
    start_time = datetime.now()
    print('\n\n')
    count_file = count_file+1
    df = pd.read_excel(f, index_col=None, header=None, usecols=None)
    print('1. Number of current file of this directory:',count_file)
    # 1. Обработка пропусков в файле
    df.iloc[1][0] = df.iloc[0][0]
    df.iloc[0][0] = None
    print('2. Count indces of current file: ',len(df.index))
    for j in range(0,len(tegs_pd)):
         for i in range(0,len(df.index)-1):
            if df.iloc[i][0] == tegs_pd.iloc[j][0]:
                df.iloc[i + 1][0] = df.iloc[i][0]
                df.iloc[i][0] = None
                break
    df.dropna(how='all', inplace=True)

    # 2. Определение количества выгруженных сообществ в каждом отдельном файле identification_community
    identification_community = pd.DataFrame(np.where(df[0]=='Дни недели'))-pd.DataFrame(np.where(df[0]=='Сравнительная таблица'))-1
    print('3. Number of communities in the current file: ', identification_community.iloc[0][0])
    # 3. копирую dataframe, чтобы сформировать "чистый" фрейм без пустых строк и правильно заполненных фреймов по характеристикам
    df_clean = df.copy()
    df_clean = df_clean.replace({np.nan: None})

    # 4. Нормализация формата вывода данных у двух признаков
    for i in range(0,len(df_clean.index) - 1):
        k = 0
        if df_clean.iloc[i][0] == tegs_extra_pd.iloc[0][0] or df_clean.iloc[i][0] == tegs_extra_pd.iloc[1][0]:
            k = identification_community.iloc[0][0]
            df_copy = pd.DataFrame(index=pd.RangeIndex(1),columns=pd.RangeIndex(k))
            for j in range(0,k):
                df_copy.iloc[0][j] = df_clean.iloc[i][j+1]
                df_clean.iloc[i][1] = None
            for j in range(0,k):
                df_clean.iloc[i][j + 2] = df_copy.iloc[0][j]
            for j in range(0,k):
                df_clean.iloc[i][k + j + 3] = df_copy.iloc[0][j]
            for j in range(0,k):
                df_clean.iloc[i][2 * k + j + 4] = df_copy.iloc[0][j]

    # 5. Определение кол-ва строк по каждой характеристике identification_difference
    iden_defer = pd.DataFrame(index = np.arange(len(tegs_full_pd.index)-1), columns = pd.RangeIndex(1))
    iden_defer = iden_defer.fillna(0)
    for i in range(1,len(tegs_full_pd.index)):
        iden_defer.iloc[i-1,:] = pd.DataFrame(np.where(df[0] == tegs_full_pd.iloc[i][0]))-pd.DataFrame(np.where(df[0] == tegs_full_pd.iloc[i-1][0])) - 1
    iden_defer = iden_defer.transpose()
    #print('4. First 10 array elements "Identification_deference" array:\n',iden_defer.iloc[0,0:10])

    # 6. Алгоритм замены повторяющихся значений в названии сообществ. В зависимости от структуру фрейма применяются различные методы
    def rename_duplicates ( iden_defer, df_clean):
        for i in range( 0, iden_defer.iloc[0][0] - 1) :
            for j in range( i + 1, iden_defer.iloc[0][0]) :
                if df_clean.iloc[i][0] == df_clean.iloc[j][0] :
                    df_clean.iloc[j][0] = str(df_clean.iloc[j][0]) + '_' + str(j)
        return df_clean

    def rename_duplicates_1 ( iden_defer, df_clean):
        for i in range( 0, iden_defer.iloc[0][0] - 1) :
            for j in range( i + 1, iden_defer.iloc[0][0]) :
                if df_clean.iloc[i][0] == df_clean.iloc[j][0] :
                    df_clean.iloc[j][0] = str(df_clean.iloc[j][0]) + '_' + str(j-1)
        return df_clean

    def rename_duplicates_2 ( df_clean ):
        for i in range( 0, len( df_clean.columns ) - 1) :
            for j in range( i + 1, len( df_clean.columns )) :
                if df_clean.iloc[0][i] == df_clean.iloc[0][j] :
                    df_clean.iloc[0][j] = str(df_clean.iloc[0][j]) + '_' + str(j-1)
        return df_clean

    def rename_duplicates_3 ( df_clean ):
        for i in range( 0, len( df_clean.columns ) - 1) :
            for j in range( i + 1, len( df_clean.columns ) ) :
                if df_clean.iloc[0][i] == df_clean.iloc[0][j] :
                    df_clean.iloc[0][j] = str(df_clean.iloc[0][j]) + '_' + str(j)
        return df_clean

    # 7. Функции для обработки различных признаков

    def func_1 ( i , j, df_clean, iden_defer, df, num_files, count_file ) :
        cut = pd.DataFrame( df_clean.iloc[ i + 1 : i + iden_defer.iloc[0][j] + 1,].to_numpy() )
        cut = rename_duplicates( iden_defer, cut )
        df = df.append( cut )
        if count_file == num_files:
            df = df.drop_duplicates(subset=[0], ignore_index=True)
            df.dropna( how = 'all', axis = 1, inplace = True )
            df.iloc[0][0] = 'Cообщества'
            df = pd.DataFrame( df.to_numpy() )
        return df

    def func_1_1 ( i , j, df_clean, iden_defer, df ) :
        cut = pd.DataFrame( df_clean.iloc[ i : i + iden_defer.iloc[0][j] + 1,].to_numpy() )
        cut = rename_duplicates( iden_defer, cut)
        df = df.append( cut )
        return df

    def func_2 ( i , j, df_clean, iden_defer, df, num_files, count_file ) :
        cut = df_clean.iloc[ i : i + iden_defer.iloc[0][j] + 1, ].to_numpy().T
        cut = pd.DataFrame( np.delete( cut, (0), axis=0 ) )
        cut.dropna( how='all', inplace=True )
        cut = rename_duplicates( iden_defer, cut)
        df = df.append(cut)
        if count_file == num_files:
            df = df.drop_duplicates(subset=[0], ignore_index=True)
        return df

    def func_3 ( i , j, df_clean, iden_defer, df, num_files, count_file ) :
        cut = pd.DataFrame( df_clean.iloc[ i : i + iden_defer.iloc[0][j] + 1, ].to_numpy() )
        cut.dropna( how='all', axis=1, inplace=True )
        cut = rename_duplicates_2( cut )
        df = df.merge( cut, on=0, how='outer' )
        if count_file == num_files:
            df = pd.DataFrame( df.T.to_numpy() )
            df = df.drop_duplicates(subset = [0], ignore_index=True)
            df = pd.DataFrame( df.T.to_numpy() )
        return df

    def func_4 (i , j, df_clean, iden_defer, df, num_files, count_file) :
        cut = pd.DataFrame( df_clean.iloc[ i : i + iden_defer.iloc[0][j] + 1, ].to_numpy().T )
        index_len = len(cut.columns) - 1

        frame_main = pd.DataFrame(index = np.arange(iden_defer.iloc[0][0] + 2), columns = pd.RangeIndex(len(cut.columns) * 3 - 2))
        frame_160 = pd.DataFrame(index = np.arange(iden_defer.iloc[0][0] + 2), columns = pd.RangeIndex(len(cut.columns)))
        frame_160_1000 = pd.DataFrame(index = np.arange(iden_defer.iloc[0][0] + 2), columns = pd.RangeIndex(len(cut.columns)))
        frame_1000 = pd.DataFrame(index = np.arange(iden_defer.iloc[0][0] + 2), columns = pd.RangeIndex(len(cut.columns)))

        value_160_1000 = pd.DataFrame( cut.iloc[ (2 + iden_defer.iloc[0][0]) : (3 + 2 * iden_defer.iloc[0][0]),].to_numpy() )
        value_1000 = pd.DataFrame( cut.iloc[ (2 + 2 * iden_defer.iloc[0][0] + 1) : (3 + 3 * iden_defer.iloc[0][0] + 1),].to_numpy() )
        value_160 = pd.DataFrame( cut.iloc[ (0) : (2 + iden_defer.iloc[0][0]),].to_numpy() )

        frame_160 = frame_160.append( value_160 )
        frame_160.dropna(how='all', inplace=True)
        for i in range(0,index_len):
            frame_160_1000.iloc[0][1 + i] = frame_160.iloc[0][1 + i]
            frame_1000.iloc[0][1 + i] = frame_160.iloc[0][1 + i]

        frame_160_1000 = frame_160_1000.append( value_160_1000 ); frame_160_1000.dropna(how='all', inplace=True)
        frame_1000 = frame_1000.append(value_1000); frame_1000.dropna(how='all', inplace=True)
        frame_main = frame_main.append(frame_160); frame_main.dropna(how='all', inplace=True)
        frame_main = pd.DataFrame( frame_main.to_numpy() )
        frame_160_1000 = pd.DataFrame( frame_160_1000.to_numpy() )
        frame_1000 = pd.DataFrame( frame_1000.to_numpy() )

        for i in range(0,iden_defer.iloc[0][0]+2):
            for j in range(0,index_len):
                frame_main.iloc[i][j + index_len + 1] = frame_160_1000.iloc[i][j + 1]
                frame_main.iloc[i][j + 2*index_len + 1] = frame_1000.iloc[i][j + 1]

        frame_main = frame_main.drop( [0,1] )
        frame_main = rename_duplicates( iden_defer, frame_main)
        df = df.append(frame_main)
        if count_file == num_files:
            df = pd.DataFrame( df.drop_duplicates(subset=[0], ignore_index=True).to_numpy() )
        return df

    def func_4_1 (i , j, df_clean, iden_defer, df) :
        cut = pd.DataFrame(df_clean.iloc[ i : i + iden_defer.iloc[0][j] + 1, ].to_numpy().T)
        index_len = len(cut.columns) - 1

        frame_main = pd.DataFrame(index = np.arange(iden_defer.iloc[0][0] + 2), columns = pd.RangeIndex(len(cut.columns) * 3 - 2))
        frame_160 = pd.DataFrame(index = np.arange(iden_defer.iloc[0][0] + 2), columns = pd.RangeIndex(len(cut.columns)))
        frame_160_1000 = pd.DataFrame(index = np.arange(iden_defer.iloc[0][0] + 2), columns = pd.RangeIndex(len(cut.columns)))
        frame_1000 = pd.DataFrame(index = np.arange(iden_defer.iloc[0][0] + 2), columns = pd.RangeIndex(len(cut.columns)))

        value_160_1000 = pd.DataFrame(cut.iloc[ (2 + iden_defer.iloc[0][0]) : (3 + 2 * iden_defer.iloc[0][0]),].to_numpy())
        value_1000 = pd.DataFrame(cut.iloc[ (2 + 2 * iden_defer.iloc[0][0] + 1) : (3 + 3 * iden_defer.iloc[0][0] + 1),].to_numpy())
        value_160 = pd.DataFrame(cut.iloc[ (0) : (2 + iden_defer.iloc[0][0]),].to_numpy())

        frame_160 = frame_160.append(value_160)
        frame_160.dropna( how = 'all', inplace = True )
        for i in range(0,index_len):
            frame_160_1000.iloc[0][1 + i] = frame_160.iloc[0][1 + i]
            frame_1000.iloc[0][1 + i] = frame_160.iloc[0][1 + i]

        frame_160_1000 = frame_160_1000.append( value_160_1000 ); frame_160_1000.dropna(how='all', inplace=True)
        frame_1000 = frame_1000.append ( value_1000 ); frame_1000.dropna(how='all', inplace=True)
        frame_main = frame_main.append( frame_160 ); frame_main.dropna(how='all', inplace=True)

        frame_main = pd.DataFrame( frame_main.to_numpy() )
        frame_160_1000 = pd.DataFrame( frame_160_1000.to_numpy() )
        frame_1000 = pd.DataFrame( frame_1000.to_numpy() )

        for i in range(0,iden_defer.iloc[0][0] + 2):
            for j in range(0,index_len):
                frame_main.iloc[i][j + index_len + 1] = frame_160_1000.iloc[i][j + 1]
                frame_main.iloc[i][j + 2 * index_len + 1] = frame_1000.iloc[i][j + 1]

        frame_main = rename_duplicates_1( iden_defer, frame_main)
        df = pd.DataFrame(df.append(frame_main).to_numpy())

        return df

    def func_5 (i , j, df_clean, iden_defer, df) :
        cut = pd.DataFrame( df_clean.iloc[ i : i + iden_defer.iloc[0][j] + 1, ].to_numpy().T )
        cut = cut.replace({np.nan: None})
        cut.dropna( how = 'all', inplace = True)
        df.dropna( how = 'all', inplace = True)
        cut = rename_duplicates( iden_defer, cut)
        df = df.append( cut )
        return df

    def func_6 (i , j, df_clean, iden_defer, df) :
        cut = pd.DataFrame( df_clean.iloc[ i : i + iden_defer.iloc[0][j] + 1, ].to_numpy() )
        cut.dropna ( how = 'all', axis = 1, inplace=True )
        cut = rename_duplicates_3 ( cut )
        df = df.append ( cut )
        return df

    def func_7 (i , j, df_clean, iden_defer, df) :
        cut = pd.DataFrame( df_clean.iloc[ i : i + iden_defer.iloc[0][j] + 1, ].to_numpy().T )
        cut.dropna( how = 'all', inplace = True )
        cut = rename_duplicates( iden_defer, cut )
        df = df.append( cut )
        return df

    # def func_8 ( i, df_clean, df ) :
    #     cut = pd.DataFrame( df_clean.iloc[i:len(df_clean.index)+1,].to_numpy() )
    #     cut.dropna( how = 'all', axis = 1, inplace = True)
    #     cut = rename_duplicates_3 ( cut )
    #     df = df.merge( cut, on = 0, how = 'outer' )
    #     if count_file == num_files:
    #         df = df.drop_duplicates(subset=[0], ignore_index=True)
    #     return df
    #
    # def func_9 ( i , df_clean, df ) :
    #     cut = pd.DataFrame( df_clean.iloc[i:len(df_clean.index)+1,].to_numpy() )
    #     cut.dropna( how = 'all', axis = 1, inplace = True)
    #     cut = rename_duplicates_2( cut )
    #     df = df.append( cut )
    #     return df

   # 8. Цикл обработки файла, где происходит разбиение его на 23 отдельных фрейма
    # 8.1. Таким образом, что каждый фрейм будет содержать информацию по каждой характеристики, НО при этом по всем сообществам из дериктории
    for i in range( 0, len( df_clean.index ) ) :
        for j in range( 0, len( tegs_full_pd.index ) ) :
            if df_clean.iloc[i][0] == tegs_full_pd.iloc[j][0] :
                if count_file > 1 :
                    if j == 0:
                        df_1 = func_1 ( i, j, df_clean, iden_defer, df_1, num_files, count_file )
                    if j == 1:
                        df_2 = func_2 ( i, j, df_clean, iden_defer, df_2, num_files, count_file )
                    if j == 2:
                        df_3 = func_2 ( i, j, df_clean, iden_defer, df_3, num_files, count_file )
                    if j == 3:
                        df_4 = func_3 ( i, j, df_clean, iden_defer, df_4, num_files, count_file )
                    if j == 4:
                        df_5 = func_3 ( i, j, df_clean, iden_defer, df_5, num_files, count_file )
                    if j == 5:
                        df_6 = func_3 ( i, j, df_clean, iden_defer, df_6, num_files, count_file )
                    if j == 6:
                        df_7 = func_3 ( i, j, df_clean, iden_defer, df_7, num_files, count_file )
                    if j == 7:
                        df_8 = func_3 ( i, j, df_clean, iden_defer, df_8, num_files, count_file )
                    if j == 8:
                        df_9 = func_3 ( i, j, df_clean, iden_defer, df_9, num_files, count_file )
                    if j == 9:
                        df_10 = func_4 ( i, j, df_clean, iden_defer, df_10, num_files, count_file )
                    if j == 10:
                        df_11 = func_4 ( i, j, df_clean, iden_defer, df_11, num_files, count_file )
                    if j == 11:
                        df_12 = func_2 ( i, j, df_clean, iden_defer, df_12, num_files, count_file )
                    if j == 12:
                        df_13 = func_2 ( i, j, df_clean, iden_defer, df_13, num_files, count_file )
                    if j == 13:
                        df_14 = func_2 ( i, j, df_clean, iden_defer, df_14, num_files, count_file )
                    if j == 14:
                        df_15 = func_2 ( i, j, df_clean, iden_defer, df_15, num_files, count_file )
                    if j == 15:
                        df_16 = func_3 ( i, j, df_clean, iden_defer, df_16, num_files, count_file )
                    if j == 16:
                        df_17 = func_3 ( i, j, df_clean, iden_defer, df_17, num_files, count_file )
                    if j == 17:
                        df_18 = func_3 ( i, j, df_clean, iden_defer, df_18, num_files, count_file )
                    if j == 18:
                        df_19 = func_4 ( i, j, df_clean, iden_defer, df_19, num_files, count_file )
                    if j == 19:
                        df_20 = func_4 ( i, j, df_clean, iden_defer, df_20, num_files, count_file )
                    if j == 20:
                        df_21 = func_2 ( i, j, df_clean, iden_defer, df_21, num_files, count_file )
                    if j == 21:
                        df_22 = func_2 ( i, j, df_clean, iden_defer, df_22, num_files, count_file )
                    # if j == 22:
                    #     df_23 = func_8 ( i, df_clean, df_23 )
                else:
                    if j == 0:
                        df_1 = func_1_1 ( i, j, df_clean, iden_defer, df_1)
                    if j == 1:
                        df_2 = func_5 ( i, j, df_clean, iden_defer, df_2)
                    if j == 2:
                        df_3 = func_5 ( i, j, df_clean, iden_defer, df_3)
                    if j == 3:
                        df_4 = func_6 ( i, j, df_clean, iden_defer, df_4)
                    if j == 4:
                        df_5 = func_6 ( i, j, df_clean, iden_defer, df_5)
                    if j == 5:
                        df_6 = func_6 ( i, j, df_clean, iden_defer, df_6)
                    if j == 6:
                        df_7 = func_6 ( i, j, df_clean, iden_defer, df_7)
                    if j == 7:
                        df_8 = func_6 ( i, j, df_clean, iden_defer, df_8)
                    if j == 8:
                        df_9 = func_6 ( i, j, df_clean, iden_defer, df_9)
                    if j == 9:
                        df_10 = func_4_1 ( i, j, df_clean, iden_defer, df_10 )
                    if j == 10:
                        df_11 = func_4_1 ( i, j, df_clean, iden_defer, df_11 )
                    if j == 11:
                        df_12 = func_7 ( i, j, df_clean, iden_defer, df_12 )
                    if j == 12:
                        df_13 = func_7 ( i, j, df_clean, iden_defer, df_13 )
                    if j == 13:
                        df_14 = func_7 ( i, j, df_clean, iden_defer, df_14 )
                    if j == 14:
                        df_15 = func_7 ( i, j, df_clean, iden_defer, df_15 )
                    if j == 15:
                        df_16 = func_6 ( i, j, df_clean, iden_defer, df_16 )
                    if j == 16:
                        df_17 = func_6 ( i, j, df_clean, iden_defer, df_17 )
                    if j == 17:
                        df_18 = func_6 ( i, j, df_clean, iden_defer, df_18 )
                    if j == 18:
                        df_19 = func_4_1 ( i, j, df_clean, iden_defer, df_19 )
                    if j == 19:
                        df_20 = func_4_1 ( i, j, df_clean, iden_defer, df_20 )
                    if j == 20:
                        df_21 = func_7 ( i, j, df_clean, iden_defer, df_21 )
                    if j == 21:
                        df_22 = func_7 ( i, j, df_clean, iden_defer, df_22 )
                    # if j == 22:
                    #     df_23 = func_9 ( i, df_clean, df_23 )
    print('4. Run time current file: ',datetime.now() - start_time)
    print('5. Algorithm running time on the current file.: ',datetime.now() - start_time1)
print('\n\n')
print('Run time: ',datetime.now() - start_time1)

Files of directory: 10






1. Number of current file of this directory: 1
2. Count indces of current file:  2778
3. Number of communities in the current file:  10
4. Run time current file:  0:00:11.615837
5. Algorithm running time on the current file.:  0:00:11.619824



1. Number of current file of this directory: 2
2. Count indces of current file:  2807
3. Number of communities in the current file:  10
4. Run time current file:  0:00:11.686133
5. Algorithm running time on the current file.:  0:00:23.307949



1. Number of current file of this directory: 3
2. Count indces of current file:  2804
3. Number of communities in the current file:  10
4. Run time current file:  0:00:12.915700
5. Algorithm running time on the current file.:  0:00:36.223649



1. Number of current file of this directory: 4
2. Count indces of current file:  2747
3. Number of communities in the current file:  10
4. Run time current file:  0:00:13.387012
5. Algorithm running time on the current file.:  0:00:49.6

##  3. Формирование общего файла с характеристиками
Фреймы без обработки или с "лёгкой" обработки, присоединяются полностью (**для части характеристик**)

In [16]:
def func_plus_type1 ( df_main, df_attached ):

    df_copy = df_attached.copy(); df_main_copy = df_main.copy()
    for i in range( 1, len( df_copy.columns ) ):
        df_copy.iloc[0][i] = str( df_copy.iloc[0][0] ) + ' ' + str( df_copy.iloc[0][i] )
    df_copy.iloc[0][0] = 'Cообщества'
    df_main_copy = df_main_copy.merge( df_copy, on = 0, how = 'outer')
    df_main_copy = pd.DataFrame( df_main_copy.to_numpy() )

    return df_main_copy

def func_plus_type2 ( df_main, df_attached ):

    df_copy = df_attached.copy(); df_main_copy = df_main.copy()
    for i in range(1,len(df_copy.columns)):
        df_copy.iloc[0][i] = str( df_copy.loc[0][0] ) + ' ' + str( df_copy.loc[0][i] ) + ' ' + str( df_copy.loc[1][i] )
    df_copy = df_copy.drop( df_copy.index[[1]] )
    df_copy.iloc[0][0] = 'Cообщества'
    df_main_copy = pd.DataFrame( df_main_copy.to_numpy() )
    df_main_copy = df_main_copy.merge( df_copy, on = 0, how = 'outer' )
    df_main_copy = pd.DataFrame( df_main_copy.to_numpy() )

    return df_main_copy

In [17]:
start_time = datetime.now()
char_2 = func_plus_type1( df_1, df_2 )
char_3 = func_plus_type1( char_2, df_3 )
char_10 = func_plus_type2( char_3, df_10 )
char_11 = func_plus_type2( char_10, df_11 )
char_12 = func_plus_type1( char_11, df_12 )
char_13 = func_plus_type1( char_12, df_13 )
char_14 = func_plus_type1( char_13, df_14 )
char_15 = func_plus_type1( char_14, df_15 )
char_19 = func_plus_type2( char_15, df_19 )
char_20 = func_plus_type2( char_19, df_20 )
char_21 = func_plus_type1( char_20, df_21 )
first_part_features = func_plus_type1( char_21, df_22 )
print('Run time: ',datetime.now() - start_time)

Run time:  0:00:00.141022


In [5]:
record_excel( first_part_features, 'first_part_features_article_test' )
print('The first part of features\n\n')
print( first_part_features.iloc[:10, 12:20] )

The first part of features


                        12        13             14             15  \
0                   Период  Тематика  Дни недели Пн  Дни недели Вт   
1  01.04.2020 - 23.08.2021    Бренды        23.8549        42.7891   
2  01.04.2020 - 23.08.2021    Бренды         7.6878         9.8419   
3  01.04.2020 - 23.08.2021    Бренды        14.8909        19.1343   
4  01.04.2020 - 23.08.2021    Бренды         8.5039        13.0733   
5  01.04.2020 - 23.08.2021    Бренды              0              0   
6  01.04.2020 - 23.08.2021    Бренды        11.8772        12.1138   
7  01.04.2020 - 23.08.2021    Бренды        13.0195        14.8222   
8  01.04.2020 - 23.08.2021    Бренды              0              0   
9  01.04.2020 - 23.08.2021    Бренды              0              0   

              16             17             18             19  
0  Дни недели Ср  Дни недели Чт  Дни недели Пт  Дни недели Сб  
1         33.356              0              0              0  
2       

##    4. Фреймы c заполнением пропусков, присоединяются только обобщённые значения для каждого сообщества:

**4.1.**  <a href="#item31"> Средним значением <a/>

**4.2.**  <a href="#item31"> Медианой <a/>

## **Методология** :
### Пропуски заполняются по правилу:
#### Были реализованы 4 функции, где заполняются пропуски значений по формуле:

#### x ( i , j ) = E_j + rand_norm ( E = 0, std = 0.55 ) × σ_j,
#### x ( i , j ) = Me_j + rand_norm ( E = 0, std = 0.55 ) × σ_j
##### где x(i,j) – пустое значение случайной величины c номером i ̇, E_j, Me_j – математическое ожидание и медиана для j сообщества, σ_j – среднеквадратическое отклонение для j сообщества.

In [18]:
pd.options.mode.chained_assignment = None # default='warn'
# 1. Функция, вытаскивающая из фрейма первую строку
def row_1_func ( df_attached ):
        first_row = df_attached.iloc[ 0,: ]
        first_row = pd.DataFrame( first_row.to_numpy() )
        return first_row
# 1.1 Функция, вытаскивающая из фрейма первые две строчки
def two_rows_func ( df_attached ):
    two_rows = df_attached.iloc[ 0:2,: ]
    two_rows = pd.DataFrame( two_rows.to_numpy() )
    return two_rows

# 2. Функция, вытаскивающая исключительно числовые значений
def normalize_func ( df_attached ):
    df_attached.dropna(how = 'all', axis = 1, inplace=True)
    df_attached = df_attached.replace( {np.nan: None} )
    df_attached = pd.DataFrame(df_attached.iloc[ 1 : len( df_attached.index ) , 1 : len( df_attached.columns ) ].to_numpy())
    return df_attached

# 3. Функция, определяющая количество пустых элементов
def count_nan ( df_attached ):
    c_nan = pd.DataFrame()
    for j in range( 0, len( df_attached.columns ) ):
        c_nan = len(df_attached) - df_attached.count()
    return c_nan

# 4. Функция, заменяющая пустые или нулевые значения по столбцам, при условии, что пропусков в этом столбце меньше половины
def insert_value ( df_attached, df_col ):
    df_attached_copy = df_attached.copy(); df_col_copy = df_col.copy()
    n = count_nan ( df_attached_copy )
    df_std = df_attached_copy.std(axis = 0)#
    # Заполняются пропуски по  правилу 3 сигм, если больше трети всех значений пропущено либо это нули, заменяю нулями
    for j in range( 0, len( df_attached_copy.columns ) ):
        if n.iloc[j] >= round( len(df_attached_copy.index)*0.33 ) :
            for i in range( 0,len( df_attached_copy.index ) ):
                df_attached_copy.iloc[i][j] = 0
        else:
            for i in range( 0,len( df_attached_copy.index ) ):
                if df_attached_copy.iloc[i][j] == 0 or df_attached_copy.iloc[i][j] is None:
                    df_attached_copy.iloc[i][j] = df_col_copy.iloc[j][0] + np.random.normal(loc=0.0, scale=0.55, size=1) * df_std.iloc[j]
    return df_attached_copy

# 5. Функция, заменяющая пропущенные значения средним, и затем вычисляется среднее
def func_mean_mean ( df_main, df_attached ):
    df_normalize = normalize_func( df_attached )
    first_row = row_1_func( df_attached ).T

    # Считаются среднее по столбцам в нормализованном фрейме
    df_normalize_copy = df_normalize.copy()
    mean_value = round( df_normalize.mean() ); mean_value = pd.DataFrame(mean_value.fillna( 0 ) )

    # Заполняются пропуски по правилу 3 сигм
    df_normalize_full = insert_value ( df_normalize_copy, mean_value )

    # Перевычисляются среднее, при условии, что пропусков нет
    final_mean_value = df_normalize_full.mean()
    final_mean_value = pd.DataFrame( final_mean_value ).T

    # В чистый фрейм добавляю к первой строке (названия сообществ) вычисленные средние значения
    char_final_mean_value  = first_row.append( final_mean_value )

    # Конкатинацией устанавливаю нормальные индексы
    char_final_mean_value = pd.concat( [char_final_mean_value], ignore_index = True)

    # Выбираю исключительно значения (средние)
    just_value = pd.DataFrame( char_final_mean_value.iloc[ 1 , 0 : ( len(char_final_mean_value.columns)-1 )].copy().to_numpy() )
    just_value = just_value.fillna( 0 )
    char_final_mean_value.iloc[1][0] = str( char_final_mean_value.iloc[0][0] ) + ' ' + 'mean mean'
    char_final_mean_value.iloc[0][0] = 'Cообщества'

    # Заполняю и округляю значения, но не для характеристики ER, потому что эта характеристика типа float
    for i in range( 1 , len( char_final_mean_value.columns ) ):
        if char_final_mean_value.iloc[1][0] == 'ER mean mean':
            char_final_mean_value.iloc[1][i] = just_value.iloc[i-1][0]
        else:
            char_final_mean_value.iloc[1][i] = round( just_value.iloc[i-1][0] )
    char_final_mean_value = char_final_mean_value.T

    df_main = df_main.merge( char_final_mean_value, on = 0, how = 'outer')
    return df_main

# 6. Функция, заменяющая пропущенные значения средним, и затем вычисляется медиана
def func_mean_median ( df_main, df_attached ):
    df_normalize = normalize_func( df_attached )
    first_row = row_1_func( df_attached ).T

    df_normalize_copy = df_normalize.copy()
    mean_value = round( df_normalize.mean() ); mean_value = pd.DataFrame(mean_value.fillna( 0 ))

    df_normalize_full = insert_value ( df_normalize_copy, mean_value )

    final_median_value = df_normalize_full.median()
    final_median_value = pd.DataFrame( final_median_value ).T

    char_final_median_value = first_row.append( final_median_value )

    char_final_median_value = pd.concat( [ char_final_median_value ], ignore_index = True)
    just_value = pd.DataFrame( char_final_median_value.iloc[ 1 , 0 : ( len(char_final_median_value.columns) - 1 ) ].copy().to_numpy() )
    just_value = just_value.fillna( 0 )

    char_final_median_value.iloc[1][0] = str( char_final_median_value.iloc[0][0] ) + ' ' + 'mean median'
    char_final_median_value.iloc[0][0] = 'Cообщества'
    for i in range( 1, len( char_final_median_value.columns ) ):
        if char_final_median_value.iloc[1][0] == 'ER mean median':
            char_final_median_value.iloc[1][i] = just_value.iloc[i-1][0]
        else:
            char_final_median_value.iloc[1][i] = round( just_value.iloc[i-1][0] )
    char_final_median_value = char_final_median_value.T

    df_main = df_main.merge( char_final_median_value, on = 0, how = 'outer')
    return df_main

# 7. Функция, заменяющая пропущенные значения медианой, и затем вычисляется медиана
def func_median_median ( df_main, df_attached ):
    df_normalize = normalize_func( df_attached )
    first_row = row_1_func( df_attached ).T

    df_normalize_copy = df_normalize.copy()
    median_value = round( df_normalize.median() ); median_value = pd.DataFrame( median_value.fillna( 0 ) )

    df_normalize_full = insert_value ( df_normalize_copy, median_value )

    final_median_value = df_normalize_full.median()
    final_median_value = pd.DataFrame( final_median_value ).T

    char_final_median_value = first_row.append( final_median_value )

    char_final_median_value = pd.concat( [ char_final_median_value ], ignore_index = True)
    just_value = pd.DataFrame( char_final_median_value.iloc[ 1 , 0 : ( len( char_final_median_value.columns ) - 1 ) ].copy().to_numpy() )
    just_value = just_value.fillna( 0 )
    char_final_median_value.iloc[1][0] = str( char_final_median_value.iloc[0][0] ) + ' ' + 'median median'
    char_final_median_value.iloc[0][0] = 'Cообщества'
    for i in range( 1, len( char_final_median_value.columns ) ):
        if char_final_median_value.iloc[1][0] == 'ER median median':
            char_final_median_value.iloc[1][i] = just_value.iloc[i-1][0]
        else:
            char_final_median_value.iloc[1][i] = round( just_value.iloc[i-1][0] )
    char_final_median_value = char_final_median_value.T

    df_main = df_main.merge( char_final_median_value, on = 0, how = 'outer' )
    return df_main

# 8. Функция, заменяющая пропущенные значения медианой, и затем вычисляется среднее
def func_median_mean ( df_main, df_attached ):
    df_normalize = normalize_func( df_attached )
    first_row = row_1_func( df_attached ).T

    df_normalize_copy = df_normalize.copy()
    median_value = round( df_normalize.median() ); median_value = pd.DataFrame( median_value.fillna( 0 ) )

    y_full_mean = insert_value ( df_normalize_copy, median_value )

    final_mean_value = y_full_mean.mean()
    final_mean_value = pd.DataFrame( final_mean_value ).T

    char_final_mean_value = first_row.append( final_mean_value )

    char_final_mean_value = pd.concat( [ char_final_mean_value ], ignore_index = True)
    just_value = pd.DataFrame( char_final_mean_value.iloc[ 1 , 0 : ( len( char_final_mean_value.columns ) - 1) ].copy().to_numpy() )
    just_value = just_value.fillna( 0 )
    char_final_mean_value.iloc[1][0] = str( char_final_mean_value.iloc[0][0] ) + ' ' + 'median mean'
    char_final_mean_value.iloc[0][0] = 'Cообщества'
    for i in range( 1, len( char_final_mean_value.columns ) ):
        if char_final_mean_value.iloc[1][0] == 'ER median mean':
            char_final_mean_value.iloc[1][i] = just_value.iloc[i-1][0]
        else:
            char_final_mean_value.iloc[1][i] = round( just_value.iloc[i-1][0] )
    char_final_mean_value = char_final_mean_value.T

    df_main = df_main.merge( char_final_mean_value , on = 0, how = 'outer')
    return df_main

# 9. Функция обработки для характеристики "Подписчики". Среднее значение.
def func_subscribers_mean ( df_main, df_attached):
    df_normalize = normalize_func( df_attached )
    first_row = two_rows_func( df_attached ).T
    first_row.iloc[0][1] = str(first_row.iloc[0][0]) + ' ' + 'mean'
    first_row.iloc[0][0] = 'Cообщества'
    mean_value = round(df_normalize.mean()); mean_value = pd.DataFrame(mean_value)
    n = count_nan ( df_normalize )
    for j in range( 0, len( first_row.index ) - 1 ):
        if n.iloc[j] >= round( len(df_normalize.index) * 0.9 ) :
            first_row.iloc[j+1][1] = 0
        else:
            first_row.iloc[j+1][1] = mean_value.iloc[j][0]
    df_main = df_main.merge( first_row , on = 0, how = 'outer')
    return df_main

# 10. Функция обработки для характеристики "Подписчики". Медиана.
def func_subscribers_median ( df_main, df_attached):
    df_normalize = normalize_func( df_attached )
    first_row = two_rows_func( df_attached ).T
    first_row.iloc[0][1] = str(first_row.iloc[0][0]) + ' ' + 'median'
    first_row.iloc[0][0] = 'Cообщества'
    median_value = round(df_normalize.median()); median_value = pd.DataFrame(median_value)
    n = count_nan ( df_normalize )
    for j in range( 0, len( first_row.index ) - 1 ):
        if n.iloc[j] >= round( len(df_normalize.index) * 0.9 ) :
            first_row.iloc[j+1][1] = 0
        else:
            first_row.iloc[j+1][1] = median_value.iloc[j][0]
    df_main = df_main.merge( first_row , on = 0, how = 'outer')
    return df_main

# 11. Функция, объединяющая все вычисленные характеристики
def connect_function( df_main, df_attached, start_time2):
    start_time = datetime.now()
    print('Start function')
    if df_attached.iloc[0][0] == 'Подписчики':
        df_main = func_subscribers_mean ( df_main,df_attached )
        df_main = func_subscribers_median ( df_main,df_attached )
    else:
        df_main = func_mean_mean( df_main,df_attached )
        df_main = func_mean_median( df_main, df_attached )
        df_main = func_median_median( df_main, df_attached )
        df_main = func_median_mean( df_main,df_attached )
        df_main = pd.DataFrame(df_main.to_numpy())
    print('1. Finish, Run time: ',datetime.now() - start_time)
    print('2. Algorithm running time on the current file.: ',datetime.now() - start_time2)

    print('\n')
    return df_main

In [19]:
start_time2 = datetime.now()
char_4 = connect_function( first_part_features, df_4, start_time2 )
char_5 = connect_function( char_4, df_5, start_time2 )
char_6 = connect_function( char_5, df_6, start_time2 )
char_7 = connect_function( char_6, df_7, start_time2 )
char_8 = connect_function( char_7, df_8, start_time2 )
char_9 = connect_function( char_8, df_9, start_time2 )
all_features = connect_function( char_9, df_16,start_time2 )
print('Run time: ',datetime.now() - start_time2)

Start function
1. Finish, Run time:  0:00:14.416899
2. Algorithm running time on the current file.:  0:00:14.416899


Start function
1. Finish, Run time:  0:00:14.005000
2. Algorithm running time on the current file.:  0:00:28.422903


Start function
1. Finish, Run time:  0:00:15.316782
2. Algorithm running time on the current file.:  0:00:43.741689


Start function
1. Finish, Run time:  0:00:14.219635
2. Algorithm running time on the current file.:  0:00:57.962328


Start function
1. Finish, Run time:  0:00:14.207380
2. Algorithm running time on the current file.:  0:01:12.170695


Start function
1. Finish, Run time:  0:00:00.263062
2. Algorithm running time on the current file.:  0:01:12.434766


Start function
1. Finish, Run time:  0:00:14.079116
2. Algorithm running time on the current file.:  0:01:26.514870


Run time:  0:01:26.515889


##    5. Сформирована матрица объекты-признаки

In [8]:
record_excel( all_features, 'all_features_article_test' )
print( all_features.iloc[:10, 251:277] )

                                                 251              252  \
0  Относительная активность по длине текста (> 10...  Лайки mean mean   
1                                                  0                0   
2                                            26.1226                0   
3                                            25.3504              879   
4                                            10.0645                0   
5                                            15.9387                0   
6                                                  0                0   
7                                            30.3514                0   
8                                               None                0   
9                                               None                0   

                 253                  254                255  \
0  Лайки mean median  Лайки median median  Лайки median mean   
1                  0                    0                  0   
2   

##    6. Добавление признаков из другого типа выгрузки (разбивка по возрастам, полу, стоимость рекламы, разбивка по странам)

In [24]:
features_other = pd.read_excel('other_features_copy.xlsx', header=None)

features_other = features_other.replace('нет данных', 0)
features_other = features_other.replace('нет', 0)
features_other = features_other.replace('да', 1)

features_other = features_other.drop_duplicates(subset=0)
all_features_copy = all_features.copy()
all_features_copy = all_features_copy.merge( features_other , on = 0, how = 'left')

In [27]:
all_features_copy = pd.DataFrame(all_features_copy.to_numpy())
all_features_copy = all_features_copy.replace({np.nan: 0})
k = len(all_features_copy.index)
for i in range(1, k):
    # k = k - 1
    if all_features_copy.iloc[i][15] == 0 or all_features_copy.iloc[i][280] == 0 or all_features_copy.iloc[i][1] == 0:
        all_features_copy = all_features_copy.drop(all_features_copy.index[[i]])
        # k = k - 1
        # all_features_copy.drop(labels = [i],axis = 1, inplace = True)
all_features_copy = all_features_copy.replace({np.nan: 0})
all_features_copy = pd.DataFrame(all_features_copy.to_numpy())

In [28]:
record_excel( all_features_copy, 'all_features_article_3750_3850' )



